In [1]:
import pandas as pd
import numpy as np
import math

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.figure_factory as ff

from IPython.display import display, HTML

init_notebook_mode(connected=True)

reg_survey_complete = pd.read_excel("Shendi_baseline_survey_for_CC_project_in_Sudan_results_20171224_utf8_villages_corregidas.xlsx", encoding = 'utf_8')

# <center>Baseline survey results for the locality of Shendi </center>
### <center>*Climate Change and Livelihoods situation of 15 villages located in Shendi Locality, River Nile State*</center>

In [2]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

#bc_wages.head()
#reg_csv.info()
#bc_wages.describe()

In [3]:
# Functions

# Get ocurrences of each value for a multiple_select field
def get_ocurrences(df, column_name):
    values_sums = df.groupby(column_name).size()
    values_names = reduce(lambda x, y: x + y,map(lambda x: x.split(), values_sums.index.values))
    result = pd.Series()
    for i in values_names:
        result[i] = values_sums.loc[[i in idx for idx in values_sums.index]].sum()
    return result

def get_percentiles_values(sorted_serie, percentiles):
    if len(percentiles) == 0:
        return []
    else:
        quantity = sorted_serie.sum() * percentiles[0]
        for i in sorted_serie.index:
            if sorted_serie[i] >= quantity:
                return [i] + get_percentiles_values(sorted_serie, percentiles[1:])
            else:
                quantity -= sorted_serie[i]

def get_percentiles(sorted_serie, values):
    if len(values) == 0:
        return []
    else:
        result = 0
        for i in sorted_serie.index:
            if values[0] == i:
                return [float(result + sorted_serie[i]) / sorted_serie.sum()] + get_percentiles(sorted_serie, values[1:])
            elif values[0] < i:
                return [float(result) / sorted_serie.sum()] + get_percentiles(sorted_serie, values[1:])
            else:
                result += sorted_serie[i]
        return [1]

In [4]:
## File preprocessing
#Deleting 'demography:', 'house_assets:', 'incomes:', 'climate_change:' and those prefix which not provide useful info.

col_names = reg_survey_complete.columns
col_names = col_names.map(lambda x: x[11:] if x.startswith("demography:") else x)
col_names = col_names.map(lambda x: x[19:] if x.startswith("responding_section:") else x)
col_names = col_names.map(lambda x: x[20:] if x.startswith("responding_section2:") else x)
col_names = col_names.map(lambda x: x[13:] if x.startswith("house_assets:") else x)
col_names = col_names.map(lambda x: x[8:] if x.startswith("incomes:") else x)
col_names = col_names.map(lambda x: x[15:] if x.startswith("climate_change:") else x)
col_names = col_names.map(lambda x: x[11:] if x.startswith("intro_data:") else x)
col_names = col_names.map(lambda x: x[:11] + x[16:] if x.startswith("respondant_page:") else x)
col_names = col_names.map(lambda x: x[16:] if x.startswith("general_hh_data:") else x)
col_names = col_names.map(lambda x: x[6:] if x.startswith("males:") else x)
col_names = col_names.map(lambda x: x[8:] if x.startswith("females:") else x)
col_names = col_names.map(lambda x: x[11:] if x.startswith("importance:") else x)
col_names = col_names.map(lambda x: x[19:] if x.startswith("percentage_incomes:") else x)
col_names = col_names.map(lambda x: x[:10] + x[16:] if x.startswith("agri_gard:agri") else x)
col_names = col_names.map(lambda x: x[:10] + x[20:] if x.startswith("livestock:livestock_") else x)
col_names = col_names.map(lambda x: x[:10] + x[21:] if x.startswith("livestock:livestock") else x)
col_names = col_names.map(lambda x: x[:8] + x[17:] if x.startswith("fishing:fishing") else x)
col_names = col_names.map(lambda x: x[:7] + x[15:] if x.startswith("labour:labour") else x)
col_names = col_names.map(lambda x: x[:7] + x[14:] if x.startswith("labour:labour") else x)
col_names = col_names.map(lambda x: x[:12] + x[25:] if x.startswith("handicrafts:handicrafts") else x)
col_names = col_names.map(lambda x: x[:9] + x[19:] if x.startswith("forestry:forestry") else x)
col_names = col_names.map(lambda x: x[:6] + x[13:] if x.startswith("other:other") else x)
col_names = col_names.map(lambda x: x[7:] if x.startswith("coping:") else x)
col_names = col_names.map(lambda x: x[4:] if x.startswith(("cc1:", "cc2:", "cc3:", "ews:")) else x)
col_names = col_names.map(lambda x: x[10:] if x.startswith("renewable:") else x)
col_names = col_names.map(lambda x: x[12:] if x.startswith("agriculture_cc_effects:") else x)
col_names = col_names.map(lambda x: x[10:] if x.startswith("livestock_cc_effects:") else x)

In [5]:
# Replace ':' with '_' for compatibility reasons.
col_names = col_names.map(lambda x: x[:14] + '_' + x[15:] if x.startswith("geocoordinates:") else x)
col_names = col_names.map(lambda x: x[:9] + '_' + x[10:] if x.startswith("agri_gard:") else x)
col_names = col_names.map(lambda x: x[:9] + '_' + x[10:] if x.startswith("livestock:") else x)
col_names = col_names.map(lambda x: x[:7] + '_' + x[8:] if x.startswith("fishing:") else x)
col_names = col_names.map(lambda x: x[:6] + '_' + x[7:] if x.startswith("labour:") else x)
col_names = col_names.map(lambda x: x[:11] + '_' + x[12:] if x.startswith("handicrafts:") else x)
col_names = col_names.map(lambda x: x[:8] + '_' + x[9:] if x.startswith("forestry:") else x)
col_names = col_names.map(lambda x: x[:5] + '_' + x[6:] if x.startswith("other:") else x)
col_names = col_names.map(lambda x: x[:10] + '_' + x[11:] if x.startswith("cc_effects:") else x)
col_names = col_names.map(lambda x: x[:4] + '_' + x[5:] if x.startswith("meta:") else x)

col_names = col_names.map(lambda x: 'livestock_lost_goat' if x == 'livestock_lost_goats' else x)

print col_names.values

reg_survey_complete.columns = col_names

[u'deviceid' u'surveyor_name' u'number' u'village'
 u'geocoordinates_Latitude' u'geocoordinates_Longitude'
 u'geocoordinates_Altitude' u'geocoordinates_Accuracy' u'respondant_gender'
 u'respondant_age' u'respondant_marital_status' u'respondant_head_hh'
 u'education' u'handicapped_member' u'number_wifes' u'number_wifes_aux'
 u'male_infant' u'male_child' u'male_child_school' u'male_teenager'
 u'male_teenager_school' u'male_adult' u'male_elderly' u'female_infant'
 u'female_child' u'female_child_school' u'female_teenager'
 u'female_teenager_school' u'female_adult' u'female_elderly'
 u'family_males' u'family_females' u'family_total' u'family_confirm'
 u'house_material' u'belongings' u'kitchen_type' u'src_incomes' u'imp_agri'
 u'imp_garden' u'imp_livestock' u'imp_fishing' u'imp_labour'
 u'imp_forestry' u'imp_handcrafts' u'imp_other' u'perc_incomes_agri'
 u'perc_incomes_garden' u'perc_incomes_livestock' u'perc_incomes_fishing'
 u'perc_incomes_labour' u'perc_incomes_forestry' u'perc_incomes_ha

In [6]:
# Processing NaN and void values

values = {
    'respondant_marital_status': 'no_answer',
    'house_material': 'no_answer',
    'kitchen_type': 'no_answer',
    'src_incomes': 'none',
    'use_ews': 'no_answer',
    'renewable_knowledge': 'no_answer',
    'solar_energy': 'no_answer',
    'livestock_selling': 'no_answer',
    'livestock_social': 'no_answer',
    'livestock_milk': 'no_answer',
    'livestock_meat': 'no_answer',
    'livestock_savings': 'no_answer',
    'livestock_number_goat': 0,
    'livestock_number_sheep': 0,
    'livestock_number_camel': 0,
    'livestock_number_cattle': 0,
    'livestock_number_chicken': 0,
    'labour_participants': 'no_answer',
    'labour_sector': '',
    'agri_gard_water_harv_farmlands': 0,
    'agri_gard_land_quantity': 0,
    'cc_effects_sand_affected_farmlands': 0
}
reg_survey_complete.fillna(value=values, inplace=True)

In [7]:
# Creating villages dict
villages = reg_survey_complete['village'].unique()
villages_table = pd.read_excel("villages_shendi.xlsx")
# Villages dictionary
villages_dict = dict()
#villages_dict['ar'] = villages_table.set_index('survey_code').to_dict()['Arabic name']
ar_dict = villages_table.set_index('survey_code').to_dict()['Arabic name']
keys = filter(lambda k: not math.isnan(k), ar_dict)
aux = {}
for i in keys:
    aux[i] = ar_dict[i]
villages_dict['ar'] = aux
en_dict = villages_table.set_index('survey_code').to_dict()['English transcription']
keys = filter(lambda k: not math.isnan(k), en_dict)
aux = {}
for i in keys:
    aux[i] = en_dict[i]
villages_dict['en'] = aux
#print repr(villages_dict).decode('unicode_escape')
print villages_dict
def reverse_dict(dic, a):
    for i in dic.keys():
        if dic[i] == a:
            return i
    return float('nan')
reg_survey_complete['village'] = reg_survey_complete.village.map((lambda x: villages_dict['en'][reverse_dict(villages_dict['ar'], x)]))


{'ar': {1.0: u'\u0627\u0644\u0628\u0648\u0627\u0644\u064a\u062f \u0627\u0644\u0633\u062a\u064a\u0631', 2.0: u'\u0628\u0626\u0631 \u0627\u0644\u0634\u064a\u0646', 3.0: u'\u0627\u0644\u0642\u0647\u064a\u062f \u0627\u0644\u062e\u0644\u0648\u0627\u062a', 4.0: u'\u0628\u0626\u0631 \u0627\u0644\u064a\u0630\u064a\u062f', 5.0: u'\u0627\u0645 \u0633\u0643\u0631\u0629', 6.0: u'\u0627\u0645 \u0639\u0634\u064a\u0631\u0629 \u0627\u0644\u063a\u0631\u0628\u064a\u0629', 7.0: u'\u0627\u0645 \u0639\u0634\u064a\u0631\u0629 \u0627\u0644\u0634\u0631\u0642\u064a\u0629', 8.0: u'\u0628\u0626\u0631 \u0627\u0644\u0633\u0642\u064a\u062f\u064a ', 9.0: u'\u0628\u0626\u0631 \u0627\u0644\u0633\u0644\u0645\u0627\u0646\u064a\u0629', 10.0: u'\u0628\u0626\u0631 \u0627\u0644\u0641\u0642\u064a\u0631\u0629 ', 11.0: u'\u0628\u0626\u0631 \u0628\u0643\u0648\u0631\u064a', 12.0: u'\u0628\u0626\u0631 \u0648\u062f \u0642\u062f\u064a\u0645', 13.0: u'\u0634\u0631\u064a\u0634\u0629', 14.0: u'\u0643\u0648\u0639 \u0627\u0644\u0633\u06

In [8]:
# For answered surveys
#reg_survey = reg_survey_complete.loc[reg_survey_complete['responding'].str.match('yes')].copy()
reg_survey = reg_survey_complete.copy()

In [9]:
# Creating num of LH strategies
reg_survey['num_lh_strats'] = reg_survey['src_incomes'].map(lambda x: len(x.split()))

In [10]:
# Separate male and female
reg_survey_male = reg_survey.loc[reg_survey['respondant_gender'].str.match('male')].copy()
reg_survey_female = reg_survey.loc[reg_survey['respondant_gender'].str.match('female')].copy()

In [11]:
## Separate per HoHH gender

def cuantos(df):
    print df['deviceid'].count()

# Respondent is HH
reg_survey['married'] = reg_survey['respondant_marital_status'].str.match('married')
# Females HoHH
reg_survey_female_head = reg_survey[reg_survey['respondant_gender'].str.match('female') &
                                      reg_survey['respondant_head_hh'].str.match('yes')].copy()
cuantos(reg_survey_female_head)
# Females noHoHH
reg_survey_female_no_head = reg_survey[reg_survey['respondant_gender'].str.match('female') &
                                      reg_survey['respondant_head_hh'].str.match('no')].copy()

# Males HoHH + Female noHoHH married
reg_survey_male_head = reg_survey[reg_survey['respondant_gender'].str.match('male') &
                                      reg_survey['respondant_head_hh'].str.match('yes')].copy()

# 0
reg_survey_male_no_head = reg_survey[reg_survey['respondant_gender'].str.match('male') &
                                      reg_survey['respondant_head_hh'].str.match('no')].copy()

reg_survey_male_head = pd.concat([reg_survey_male_head,
                                  reg_survey_female_no_head[reg_survey_female_no_head['married']]])

# Add females nHoHH nM with male_adult to male_head
reg_survey_female_nHoHH_nM = reg_survey_female_no_head[~reg_survey_female_no_head['married']]
reg_survey_male_head = pd.concat([reg_survey_male_head,
                                  reg_survey_female_nHoHH_nM[reg_survey_female_nHoHH_nM['male_adult'] > 0]])

# Add females nHoHH nM without male_adult to female_head
reg_survey_female_head = pd.concat([reg_survey_female_head,
                                    reg_survey_female_nHoHH_nM[reg_survey_female_nHoHH_nM['male_adult'] == 0]])

# Add males nHoHH with male_adult to male_head
reg_survey_male_nHoHH = reg_survey[reg_survey['respondant_gender'].str.match('male') &
                                      reg_survey['respondant_head_hh'].str.match('no')].copy()
reg_survey_maleHoHH = pd.concat([reg_survey_male_head,
                                  reg_survey_male_nHoHH[reg_survey_male_nHoHH['male_adult'] > 0]])

# Add females nHoHH with male_adult to female_head
reg_survey_femHoHH = pd.concat([reg_survey_female_head,
                                  reg_survey_male_nHoHH[reg_survey_male_nHoHH['male_adult'] == 0]])


55


In [12]:
HoHH = {}
HoHH['Male HoHH'] = reg_survey_maleHoHH['deviceid'].count()
HoHH['Female HoHH'] = reg_survey_femHoHH['deviceid'].count()

HoHH = pd.Series(HoHH)

fig = {
    "layout": {
        "title": "Households according to the gender of the HoHH"
    },
    "data": [
        {
            "values": HoHH.values,
            "labels": HoHH.index.values,
            "type": "pie",
            "sort": False
        }
    ]
}

iplot(fig)

In [13]:
reg_survey_hand_yes = reg_survey.loc[reg_survey.handicapped_member == 'yes'].copy()
reg_survey_hand_no = reg_survey.loc[reg_survey.handicapped_member == 'no'].copy()
print reg_survey['deviceid'].count()
yes_no_number = [reg_survey_hand_yes['deviceid'].count(), reg_survey_hand_no['deviceid'].count()]
yes_no_values = ['yes', 'no']

fig = {
    "layout": {
        "title": "Households with handicapped members"
    },
    "data": [
        {
            "values": yes_no_number,
            "labels": yes_no_values,
            "type": "pie",
            "sort": False
        }
    ]
}

iplot(fig)

217


In [14]:
def create_graphs_HoHH(data_func, values_func, labels_func, sum_func, title):
    create_three_pie_graphs((reg_survey_femHoHH, reg_survey, reg_survey_maleHoHH), data_func, values_func,
                            labels_func, sum_func, title,
                            "Female Head of Household", .04,
                            "Global", .5,
                            "Male Head of Household", .94)

def create_graphs_respondent(data_func, values_func, labels_func, sum_func, title):
    create_three_pie_graphs((reg_survey_female, reg_survey, reg_survey_male), data_func, values_func,
                            labels_func, sum_func, title,
                            "Female Respondent", .07,
                            "Global", .5,
                            "Male Respondent", .91)

def create_pie_graph(serie, title):
    fig = {
        "layout": {
            "title": title
        },
        "data": [
            {
                "values": serie.values,
                "labels": serie.index.values,
                "type": "pie",
                "sort": False
            }
        ]
    }

    iplot(fig)
    
    
def create_three_pie_graphs((df1, df2, df3), data_func, values_func,
                            labels_func, sum_func, title,
                            title1, title1_x_offset,
                            title2, title2_x_offset,
                            title3, title3_x_offset):
    data_fem = data_func(df1)
    data_global = data_func(df2)
    data_male = data_func(df3)

    fig = {
        "layout": {
            "title": title,
            "annotations": [
                {
                    "font": {
                        "size": 14
                    },
                    "showarrow": False,
                    #"text": title1 + " (" + str(sum_func(data_fem)) + ")",
                    "text": title1,
                    "x": title1_x_offset,
                    "y": 1
                },
                {
                    "font": {
                        "size": 14
                    },
                    "showarrow": False,
                    #"text": title2 + " (" + str(sum_func(data_global)) + ")",
                    "text": title2,
                    "x": title2_x_offset,
                    "y": 1
                },
                {
                    "font": {
                        "size": 14
                    },
                    "showarrow": False,
                    #"text": title3 + " (" + str(sum_func(data_male)) + ")",
                    "text": title3,
                    "x": title3_x_offset,
                    "y": 1
                }
            ]
        },
        "data": [
            {
                "values": values_func(data_fem),
                "labels": labels_func(data_fem),
                "domain": {
                    "x": [0, .32]
                },
                "type": "pie",
                "sort": False
            },
            {
                "values": values_func(data_global),
                "labels": labels_func(data_global),
                "domain": {
                    "x": [.34, .64]
                },
                "type": "pie",
                "sort": False
            },
            {
                "values": values_func(data_male),
                "labels": labels_func(data_male),
                "domain": {
                    "x": [.66, .98]
                },
                "type": "pie",
                "sort": False
            }
        ]
    }

    iplot(fig)

def create_stacked_bars_graph((df1, df2), data_func, x_func, y_func, title,
                              title_df1, title_df2):
    data_global = data_func(pd.concat([df1, df2]))
    data_fem = data_func(df1)
    data_male = data_func(df2)

    fig = {
        "layout": {
            "title": title,
            "xaxis": {
                "tickangle": -45
            },
            "barmode": "stack",
            "width": 700,
            "annotations": []
        },
        "data": [
            {
                "x": x_func(data_fem),
                "y": y_func(data_fem),
                "type": "bar",
                "name": title_df1,
                "marker": {
                    "color": 'rgb(225,225,258)',
                    "line": {
                        "color": 'rgb(0,0,192)',
                        "width": 1.5
                    }
                },
            },
            {
                "x": x_func(data_male),
                "y": y_func(data_male),
                "type": "bar",
                "name": title_df2,
                "marker": {
                    "color": 'rgb(258,225,225)',
                    "line": {
                        "color": 'rgb(192,0,0)',
                        "width": 1.5
                    }
                },
            }
        
        ]
    }


#    for i in range(0, len(kitchen_type)):
#        fig["layout"]["annotations"].append(
#            {
#                 "x": map(lambda x: x.capitalize(), kitchen_perc.index.values)[i],
#                 "y": kitchen_perc.values[i] + 1.5,
#                "x": pd.concat([df1, df2])[i],
#                "y": y_func(data_global)[i] + 1.5,
#                "text": str(data_global.values[i]) + "%",
#                "font": dict(
#                    family='Arial',
#                    size=14,
#                    color='rgba(0, 0, 100, 1)'
#                ),
#                "showarrow": False,
#    
#            }
#        )
#
#
    iplot(fig)
    

def plot_wealth_criteria(groupby_data, values, title):
    groupby_data = groupby_data * 100 / groupby_data.sum()
    poor = groupby_data[groupby_data.index <= values[0]]
    medium = groupby_data[groupby_data.index > values[0]]
    medium = medium[medium.index <= values[1]]
    rich = groupby_data[groupby_data.index > values[1]]
    rich = rich[rich.index <= values[2]]
    print medium
    
    fig_distribution = {
        "layout": {
            "title": title,
            "yaxis": {
                "ticksuffix": "%"
            },
            "width": 700,
            "annotations": []
        },
        "data": [
            {
                "x": poor.index.values,
                "y": poor.values,
                "type": "bar",
                "name": "Very poor (" + str(round(poor.values.sum(), 1)) + "%)" ,
                "marker": {
                    "color": 'rgb(214,39,40)',
                }
            },
            {
                "x": medium.index.values,
                "y": medium.values,
                "type": "bar",
                "name": "Poor (" + str(round(medium.values.sum(), 1)) + "%)",
                "marker": {
                    "color": 'rgb(255,127,14)',
                }
            },
            {
                "x": rich.index.values,
                "y": rich.values,
                "type": "bar",
                "name": "Medium (" + str(round(rich.values.sum(), 1)) + "%)",
                "marker": {
                    "color": 'rgb(44,160,44)',
                }
            }
        ]
    }
    
    iplot(fig_distribution)


## General data

Number of Surveys:

In [15]:
print "Total:" + str(reg_survey_complete['deviceid'].count())
print ""
total_surveys = reg_survey['deviceid'].count()
print "Responding:" + str(total_surveys)
print ""

Total:217

Responding:217



### Respondents per gender and village

In [16]:
surveys_village = reg_survey.groupby('village').size()
surveys_village_male = reg_survey_male.groupby('village').size()
surveys_village_female = reg_survey_female.groupby('village').size()
display(surveys_village_male)
respondents_gender = pd.DataFrame(surveys_village, columns= ['Total'])
respondents_gender['Male'] = surveys_village_male
respondents_gender['Male %'] = (surveys_village_male*100 / surveys_village).round(2)
respondents_gender['Female'] = surveys_village_female
respondents_gender['Female %'] = (surveys_village_female*100 / surveys_village).round(2)
respondents_gender = respondents_gender.reindex(['Male', 'Male %', 'Female', 'Female %', 'Total'], axis=1)
respondents_gender.loc['Total', 'Total'] = respondents_gender['Total'].sum()
respondents_gender.loc['Total', 'Male'] = respondents_gender['Male'].sum()
respondents_gender.loc['Total', 'Female'] = respondents_gender['Female'].sum()
respondents_gender.fillna({'Male': 0, 'Female': 0}, inplace=True)
respondents_gender.loc['Total', 'Male %'] = round(100*respondents_gender.loc['Total', 'Male'] / respondents_gender.loc['Total', 'Total'], 2)
respondents_gender.loc['Total', 'Female %'] = round(100*respondents_gender.loc['Total', 'Female'] / respondents_gender.loc['Total', 'Total'], 2)
respondents_gender['Male'] = respondents_gender['Male'].astype(int)
respondents_gender['Female'] = respondents_gender['Female'].astype(int)
respondents_gender['Total'] = respondents_gender['Total'].astype(int)

display(respondents_gender)

village
Bir Bakori              7
Bir Elfageera           1
Bir Elsegedi            5
Bir Elselemaniya        6
Bir Elsheen             3
Bir Elyased W & E       4
Bir Wad Gadeem          5
Elbwaleed Elsetayer     6
Elgiheed Elkhalwat      5
Koa Elsereh            11
Om Esheera east        26
Om Esheera west        22
Om Sokara               6
Shiresha               15
Wadi Eldan              6
dtype: int64

,Male,Male %,Female,Female %,Total
village,,,,,
Bir Bakori,7,46.67,8,53.33,15
Bir Elfageera,1,16.67,5,83.33,6
Bir Elsegedi,5,41.67,7,58.33,12
Bir Elselemaniya,6,54.55,5,45.45,11
Bir Elsheen,3,60.00,2,40.00,5
Bir Elyased W & E,4,66.67,2,33.33,6
Bir Wad Gadeem,5,35.71,9,64.29,14
Elbwaleed Elsetayer,6,35.29,11,64.71,17
Elgiheed Elkhalwat,5,33.33,10,66.67,15


### HHs Composition

In [17]:
surveys_village_maleHoHH = reg_survey_maleHoHH.groupby('village').size()
surveys_village_femHoHH = reg_survey_femHoHH.groupby('village').size()
HoHH_gender = pd.DataFrame(surveys_village, columns= ['Total'])
HoHH_gender['Male'] = surveys_village_maleHoHH
HoHH_gender['Male %'] = (surveys_village_maleHoHH*100 / surveys_village).round(2)
HoHH_gender['Female'] = surveys_village_femHoHH
HoHH_gender['Female %'] = (surveys_village_femHoHH*100 / surveys_village).round(2)
HoHH_gender = HoHH_gender.reindex(['Male', 'Male %', 'Female', 'Female %', 'Total'], axis=1)
HoHH_gender.loc['Total', 'Total'] = HoHH_gender['Total'].sum()
HoHH_gender.loc['Total', 'Male'] = HoHH_gender['Male'].sum()
HoHH_gender.loc['Total', 'Male %'] = round(100*HoHH_gender.loc['Total', 'Male'] / HoHH_gender.loc['Total', 'Total'], 2)
HoHH_gender.loc['Total', 'Female'] = HoHH_gender['Female'].sum()
HoHH_gender.loc['Total', 'Female %'] = round(100*HoHH_gender.loc['Total', 'Female'] / HoHH_gender.loc['Total', 'Total'], 2)
HoHH_gender.fillna({'Male': 0, 'Female': 0}, inplace=True)
HoHH_gender['Male'] = HoHH_gender['Male'].astype(int)
HoHH_gender['Female'] = HoHH_gender['Female'].astype(int)
HoHH_gender['Total'] = HoHH_gender['Total'].astype(int)

display(HoHH_gender)

,Male,Male %,Female,Female %,Total
village,,,,,
Bir Bakori,11,73.33,4,26.67,15
Bir Elfageera,2,33.33,4,66.67,6
Bir Elsegedi,6,50.00,6,50.00,12
Bir Elselemaniya,7,63.64,4,36.36,11
Bir Elsheen,4,80.00,1,20.00,5
Bir Elyased W & E,5,83.33,1,16.67,6
Bir Wad Gadeem,12,85.71,2,14.29,14
Elbwaleed Elsetayer,9,52.94,8,47.06,17
Elgiheed Elkhalwat,9,60.00,6,40.00,15


In [18]:
print reg_survey_femHoHH['deviceid'].count()
print reg_survey_maleHoHH['deviceid'].count()

60
157


In [19]:
create_graphs_HoHH(lambda x: x.groupby('education').size(),
                 lambda x: x.values,
                 lambda x: x.index.values,
                   lambda x: x.sum(),
                 "Education level of the head of household")


In [20]:
#hey = reg_survey.groupby(['village', 'house_material']).size()

#house_materials_df = pd.DataFrame(index=house_materials_index, columns=villages, data=[house_materials_general])
#print house_materials_df

create_graphs_HoHH(lambda x: pd.crosstab(x.village, x.house_material),
                 lambda x: x.sum().values,
                 lambda x: x.columns.values,
                   lambda x: x.sum().sum(),
                 "House materials")

create_pie_graph(reg_survey.groupby('house_material').size(), "House materials")

fig_house_materials2 = {
    "layout": {
        "title": {
            "font": {
                    "size": 20
            },
            "text": "House materials"
        }
    },
    "data": []
}

house_materials_df = pd.crosstab(reg_survey.village, reg_survey.house_material)
#print range(len(villages_dict['en'].keys())-1)
#print house_materials_df
#for i in range(len(villages_dict['en'].keys())):
#    x = .2*(i%5)
#    y = 0.35*(i/5)
#    fig_house_materials2['data'].append(
#        {
#
#            "values": house_materials_df.iloc[i-1].values,
#            "labels": house_materials_df.columns.values,
#             "domain": {
#                "x": [x, x+.18],
#                 "y": [y, y+.3]
#            },
#            "type": "pie",
#            "sort": False
#        })
#
#iplot(fig_house_materials2)

###  Mean Household size

In [21]:
mean_hh_size = reg_survey.family_total.mean()
print str(mean_hh_size) + " persons per household"

4.01382488479 persons per household


## Climate change awareness

In [22]:
create_graphs_respondent(lambda x: x.groupby('cc_knowledge').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Households declaring CC knowledge")

In [23]:
create_graphs_respondent(lambda x: x.groupby('taking_action_cc').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Households declaring taking actions against CC")

#### EWS awareness

In [24]:
create_graphs_respondent(lambda x: x.groupby('ews_knowledge').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Households declaring EWS knowledge")

In [25]:
create_graphs_respondent(lambda x: x.groupby('use_ews').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Households using EWS information")

#### Renewable energy awareness

In [26]:
create_graphs_respondent(lambda x: x.groupby('renewable_knowledge').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Households declaring renewable energies knowledge")

In [27]:
create_graphs_respondent(lambda x: x.groupby('solar_energy').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Households using solar energies")

#### Cooking facilities

In [28]:
kitchen_type = reg_survey.groupby('kitchen_type').size()
kitchen_type_male = get_ocurrences(reg_survey_maleHoHH, 'kitchen_type').sort_values(ascending = False)
kitchen_type_female = get_ocurrences(reg_survey_femHoHH, 'kitchen_type').sort_values(ascending = False)
kitchen = pd.crosstab(reg_survey.village, reg_survey.kitchen_type)
#display(kitchen)

kitchen_perc_male = 100*kitchen_type_male/total_surveys
kitchen_perc_female = 100*kitchen_type_female/total_surveys
kitchen_perc = 100*kitchen_type/total_surveys
kitchen_perc = kitchen_perc.round(2)
#print kitchen_perc

create_stacked_bars_graph((reg_survey_femHoHH, reg_survey_maleHoHH),
                              lambda x: 100 * get_ocurrences(x, 'kitchen_type').sort_values(ascending = False) / total_surveys,
                              lambda x: map(lambda y: y.capitalize(), x.index.values),
                              lambda x: x.values,
                              "Cooking techniques",
                              "Female Head of Household",
                              "Male Head of Household")

## Livelihoods

Number of households practicing each livelihood

In [29]:
#reg_survey.loc[reg_survey['src_incomes'].str.contains("agriculture"), 'agriculture'] = 'yes'
#reg_survey.loc[reg_survey['src_incomes'].str.contains("livestock"), 'livestock'] = 'yes'
#reg_survey.loc[reg_survey['src_incomes'].str.contains("labour"), 'labour'] = 'yes'
#importance = reg_survey.groupby(['agriculture', 'livestock', 'labour'])['perc_incomes_agri'].agg('mean')

#agric = reg_survey[reg_survey['src_incomes'].str.contains("agriculture")]['deviceid'].count()

src_incomes_sums = reg_survey.groupby('src_incomes').size()
lh_values = get_ocurrences(reg_survey, 'src_incomes')
lh_types = pd.DataFrame({"Households": lh_values})
#lh_types["Percentage"] = lh_types.Households.map(lambda x: str(round(100*x/total_surveys))) + " %"

print ""
print lh_values.sort_values(ascending = False)


labour         104
livestock      100
agriculture     57
other           39
handicrafts      3
forestry         1
dtype: int64


In [30]:
lh_perc = 100*lh_values/total_surveys
lh_perc = lh_perc.round(2)

create_stacked_bars_graph((reg_survey_femHoHH, reg_survey_maleHoHH),
                              lambda x: 100 * get_ocurrences(x, 'src_incomes') / total_surveys,
                              lambda x: map(lambda y: y.capitalize(), x.index.values),
                              lambda x: x.values,
                              "Livelihoods Strategies",
                              "Female Head of Household",
                              "Male Head of Household")

create_stacked_bars_graph((reg_survey_female, reg_survey_male),
                              lambda x: 100 * get_ocurrences(x, 'src_incomes') / total_surveys,
                              lambda x: map(lambda y: y.capitalize(), x.index.values),
                              lambda x: x.values,
                              "Livelihoods Strategies",
                              "Female respondent",
                              "Male respondent")

livelihoods_strategies = {
    "layout": {
        "title": "Livelihoods Strategies",
        "xaxis": {
            "tickangle": -45
        },
        "width": 700,
        "annotations": []
    },
    "data": [
        {
            "x": map(lambda x: x.capitalize(), lh_perc.index.values),
            "y": lh_perc.values,
            "type": "bar",
            "name": "Livelihoods Strategies",
            "marker": {
                "color": 'rgb(258,225,225)',
                "line": {
                    "color": 'rgb(192,0,0)',
                    "width": 1.5
                }
            },
        }
    ]
}


for i in range(0, len(lh_values)):
    livelihoods_strategies["layout"]["annotations"].append(
        {
            "x": map(lambda x: x.capitalize(), lh_perc.index.values)[i],
            "y": lh_perc.values[i] + 1.5,
            "text": str(lh_perc.values[i]) + "%",
            "font": dict(
                family='Arial',
                size=14,
                color='rgba(100, 0, 0, 1)'
            ),
            "showarrow": False,

        }
    )

iplot(livelihoods_strategies)

In [31]:
lh_values_hand = get_ocurrences(reg_survey_hand_yes, 'src_incomes')
lh_perc_hand = 100*lh_values_hand/reg_survey_hand_yes['deviceid'].count()
lh_perc_hand = lh_perc_hand.round(2)

livelihoods_strategies_hand = {
    "layout": {
        "title": "Livelihoods Strategies in HH with dependent members",
        "xaxis": {
            "tickangle": -45
        },
        "width": 700,
        "annotations": []
    },
    "data": [
        {
            "x": map(lambda x: x.capitalize(), lh_perc_hand.index.values),
            "y": lh_perc_hand.values,
            "type": "bar",
            "name": "Livelihoods Strategies",
            "marker": {
                "color": 'rgb(258,225,225)',
                "line": {
                    "color": 'rgb(192,0,0)',
                    "width": 1.5
                }
            },
        }
    ]
}


for i in range(0, len(lh_values_hand)):
    livelihoods_strategies_hand["layout"]["annotations"].append(
        {
            "x": map(lambda x: x.capitalize(), lh_perc_hand.index.values)[i],
            "y": lh_perc_hand.values[i] + 1.2,
            "text": str(lh_perc_hand.values[i]) + "%",
            "font": dict(
                family='Arial',
                size=14,
                color='rgba(100, 0, 0, 1)'
            ),
            "showarrow": False,

        }
    )

iplot(livelihoods_strategies_hand)

Main livelihoods per household

In [32]:
print src_incomes_sums.sort_values(ascending = False)
reg_survey['num_lh_strats'] = reg_survey['src_incomes'].map(lambda x: len(x.split()))
create_graphs_HoHH(lambda x: x.groupby('num_lh_strats').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Number of livelihoods Strategies")
create_graphs_respondent(lambda x: x.groupby('num_lh_strats').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Number of livelihoods Strategies")

src_incomes
labour                               75
livestock                            41
agriculture livestock                29
other                                22
livestock labour                     13
agriculture livestock labour          6
livestock other                       5
agriculture labour                    5
agriculture                           5
agriculture other                     4
labour other                          3
agriculture livestock other           3
agriculture livestock handicrafts     2
agriculture labour other              2
handicrafts                           1
agriculture livestock forestry        1
dtype: int64


In [33]:
reg_survey_hand_yes['num_lh_strats'] = reg_survey_hand_yes['src_incomes'].map(lambda x: len(x.split()))
create_pie_graph(reg_survey_hand_yes.groupby('num_lh_strats').size(), "Number livelihoods strategies in HH with dependent members")

- 277 out of 399 hh, i.e. 69.42%, lives only on livestock, forestry or both.

### Livestock

Number of interviewed households owning each type of livestock

In [34]:
livestock_set = reg_survey.loc[reg_survey['src_incomes'].str.contains("livestock")]
livestock_set_female = reg_survey_female.loc[reg_survey_female['src_incomes'].str.contains("livestock")]
livestock_set_femHoHH = reg_survey_femHoHH.loc[reg_survey_femHoHH['src_incomes'].str.contains("livestock")]

participants = livestock_set.groupby('livestock_participants').size()
participants_goats = livestock_set.loc[livestock_set['livestock_type'].str.contains("goat")].groupby('livestock_participants').size()
#print participants_goats

lvstk_types = ['camel', 'cattle', 'goat', 'sheep', 'chicken']

livestock_women = livestock_set.loc[livestock_set['livestock_participants'] == "women"]
type_women =  livestock_women.groupby('livestock_type').size()
#livestock_set = livestock_set.copy()
for i in lvstk_types:
    livestock_set[i + '_reduced'] = livestock_set['livestock_last_year_' + i] / (livestock_set['livestock_number_' + i] + livestock_set['livestock_number_' + i])
#print ("Types for women")
#print(type_women)
#print livestock_set.groupby('camel_reduced').size()
goats = livestock_set['livestock_number_goat'].dropna()
#goats_female = livestock_set_female['livestock_number_goat'].dropna()
goats_femHoHH = livestock_set_femHoHH['livestock_number_goat'].dropna()

sheeps = livestock_set['livestock_number_sheep'].dropna()
#sheeps_female = livestock_set_female['livestock_number_sheep'].dropna()
sheeps_femHoHH = livestock_set_femHoHH['livestock_number_sheep'].dropna()

cattle = livestock_set['livestock_number_cattle'].dropna()
#cattle_female = livestock_set_female['livestock_number_cattle'].dropna()
cattle_femHoHH = livestock_set_femHoHH['livestock_number_cattle'].dropna()

chickens = livestock_set['livestock_number_chicken'].dropna()
#chickens_female = livestock_set_female['livestock_number_chicken'].dropna()
chickens_femHoHH = livestock_set_femHoHH['livestock_number_chicken'].dropna()

camels = livestock_set['livestock_number_camel'].dropna()

livestock_set.groupby('livestock_training').size()


livestock_training
no     98
yes     1
dtype: int64

In [35]:
livestock_set['livestock_number_gosh'] = livestock_set['livestock_number_goat'] + livestock_set['livestock_number_sheep']

print livestock_set['livestock_number_gosh'].sum()
print livestock_set['livestock_number_goat'].mean()
print livestock_set.loc[livestock_set.livestock_number_goat <= 50]['livestock_number_goat'].mean()
print livestock_set['livestock_number_sheep'].mean()
print livestock_set.loc[livestock_set.livestock_number_sheep <= 80]['livestock_number_sheep'].mean()

types_lvstck = get_ocurrences(livestock_set, 'livestock_type')

def score_gs(gs):
    if (gs<= 5):
        result = 0
    elif (gs <= 20):
        result = 1
    elif (gs <= 50):
        result = 2
    elif (gs <= 150):
        result = 3
    else:
        result = 5
    return result

def score_camel(gs):
    if (gs<= 0):
        result = 0
    elif (gs <= 3):
        result = 1
    elif (gs <= 6):
        result = 2
    elif (gs <= 15):
        result = 3
    else:
        result = 5
    return result

def score_cattle(gs):
    if (gs<= 0):
        result = 0
    elif (gs <= 2):
        result = 1
    elif (gs <= 5):
        result = 2
    elif (gs <= 15):
        result = 3
    else:
        result = 5
    return result

def score_wealth_lvstck(df):
    sum_gs = df['livestock_number_gosh'].map(score_gs)
    sum_camel = df['livestock_number_camel'].map(score_camel)
    sum_cattle = df['livestock_number_cattle'].map(score_cattle)
    df['wealth_group_score'] = sum_gs + sum_camel + sum_cattle

score_wealth_lvstck(livestock_set)

5516.0
21.48
14.5052631579
33.68
14.808988764


In [36]:
perc_lvstck = 100*types_lvstck/livestock_set['deviceid'].count()
perc_lvstck = perc_lvstck.round(2)
fig_types_lvstck = {
    "layout": {
        "title": "Livestock types",
        "xaxis": {
            "tickangle": -45
        },
        "width": 700,
        "annotations": []
    },
    "data": [
        {
            "x": map(lambda x: x.capitalize(), perc_lvstck.index.values),
            "y": perc_lvstck.values,
            "type": "bar",
            "name": "Livelihoods Strategies",
            "marker": {
                "color": 'rgb(258,225,225)',
                "line": {
                    "color": 'rgb(192,0,0)',
                    "width": 1.5
                }
            }
        }
    ]
}


for i in range(0, len(types_lvstck)):
    fig_types_lvstck["layout"]["annotations"].append(
        {
            "x": map(lambda x: x.capitalize(), perc_lvstck.index.values)[i],
            "y": perc_lvstck.values[i] + 1.5,
            "text": str(perc_lvstck.values[i]) + "%",
            "font": dict(
                family='Arial',
                size=14,
                color='rgba(100, 0, 0, 1)'
            ),
            "showarrow": False,

        }
    )

iplot(fig_types_lvstck)


In [37]:
prueba = pd.crosstab(livestock_set.livestock_number_goat, livestock_set.livestock_number_sheep).stack().reset_index(name='number')
#prueba.plot.scatter('livestock_number_goats', 'livestock_number_sheep', s=prueba.number * 10)
prueba = prueba[prueba.number != 0]
#prueba = prueba[prueba.livestock_number_goat <= 100]
#prueba = prueba[prueba.livestock_number_sheep <= 100]
prueba = prueba.drop(0)
#display(prueba)
goats_vs_sheeps = {
    "layout": {
        "title": "Relation between the number of goats and sheeps",
        "xaxis": {
            "title": "Number of goats"
        },
        "yaxis": {
            "title": "Number of sheeps"
        }
    },
    "data": [
        {

            "x": prueba.livestock_number_goat,
            "y": prueba.livestock_number_sheep,
            "mode": "markers",
            "marker": {
                "size": prueba.number * 5               
            },
            "type": "scatter"
        }
    ]
}
iplot(goats_vs_sheeps)

In [38]:
prueba2 = pd.crosstab(livestock_set.livestock_number_cattle, livestock_set.livestock_number_sheep).stack().reset_index(name='number')
#prueba.plot.scatter('livestock_number_goats', 'livestock_number_sheep', s=prueba.number * 10)
prueba2 = prueba2[prueba2.number != 0]
prueba2= prueba2.drop(0)
prueba2 = prueba2[prueba2.livestock_number_cattle <= 100]
prueba2 = prueba2[prueba2.livestock_number_sheep <= 100]
#display(prueba)
cattle_vs_sheeps = {
    "layout": {
        "title": "Relation between the number of cattle and sheeps",
        "xaxis": {
            "title": "Number of cattle"
        },
        "yaxis": {
            "title": "Number of sheeps"
        }
    },
    "data": [
        {

            "x": prueba2.livestock_number_cattle,
            "y": prueba2.livestock_number_sheep,
            "mode": "markers",
            "marker": {
                "size": prueba2.number * 5               
            },
            "type": "scatter"
        }
    ]
}
iplot(cattle_vs_sheeps)

In [39]:
prueba3 = pd.crosstab(livestock_set.livestock_number_goat, livestock_set.livestock_number_cattle).stack().reset_index(name='number')
#prueba.plot.scatter('livestock_number_goats', 'livestock_number_sheep', s=prueba.number * 10)
prueba3 = prueba3[prueba3.number != 0]
prueba3= prueba3.drop(0)
prueba3 = prueba3[prueba3.livestock_number_goat <= 100]
prueba3 = prueba3[prueba3.livestock_number_cattle <= 100]
#display(prueba)
goat_vs_cattle = {
    "layout": {
        "title": "Relation between the number of cattle and goats",
        "xaxis": {
            "title": "Number of goat"
        },
        "yaxis": {
            "title": "Number of cattle"
        }
    },
    "data": [
        {

            "x": prueba3.livestock_number_goat,
            "y": prueba3.livestock_number_cattle,
            "mode": "markers",
            "marker": {
                "size": prueba3.number * 5               
            },
            "type": "scatter"
        }
    ]
}
iplot(goat_vs_cattle)

In [40]:
prueba4 = pd.crosstab(livestock_set.livestock_number_goat, livestock_set.livestock_number_camel).stack().reset_index(name='number')
#prueba.plot.scatter('livestock_number_goats', 'livestock_number_sheep', s=prueba.number * 10)
prueba4 = prueba4[prueba4.number != 0]
prueba4= prueba4.drop(0)
prueba4 = prueba4[prueba4.livestock_number_goat <= 100]
prueba4 = prueba4[prueba4.livestock_number_camel <= 100]
#display(prueba)
goat_vs_camel = {
    "layout": {
        "title": "Relation between the number of camel and sheeps",
        "xaxis": {
            "title": "Number of goat"
        },
        "yaxis": {
            "title": "Number of camel"
        }
    },
    "data": [
        {

            "x": prueba4.livestock_number_goat,
            "y": prueba4.livestock_number_camel,
            "mode": "markers",
            "marker": {
                "size": prueba4.number * 5               
            },
            "type": "scatter"
        }
    ]
}
iplot(goat_vs_camel)

In [41]:
def getColor5(value):
    if (value <=0):
        return "red"
    elif (value <= 3):
        return "orange"
    else:
        return "green"

def getWealth5(df):
    df['wealth'] = df['livestock_number_gosh'].map(score_gs) + df['livestock_number_camel'].map(score_camel)
    return df['wealth']
    
prueba5 = pd.crosstab(livestock_set.livestock_number_gosh, livestock_set.livestock_number_camel).stack().reset_index(name='number')
#prueba.plot.scatter('livestock_number_goats', 'livestock_number_sheep', s=prueba.number * 10)
prueba5 = prueba5[prueba5.number != 0]
#prueba5 = prueba5[prueba5.livestock_number_gosh <= 100]
#prueba5 = prueba5[prueba5.livestock_number_camel <= 100]
#display(prueba)
gosh_vs_camel = {
    "layout": {
        "title": "Relation between the number of camel and sheeps",
        "xaxis": {
            "title": "Number of goat"
        },
        "yaxis": {
            "title": "Number of camel"
        }
    },
    "data": [
        {

            "x": prueba5.livestock_number_gosh,
            "y": prueba5.livestock_number_camel,
            "mode": "markers",
            "marker": {
                "size": prueba5.number * 5, 
                "color": getWealth5(prueba5).map(getColor5)
            },
            "type": "scatter"
        }
    ]
}
iplot(gosh_vs_camel)

In [42]:
def getColor6(value):
    if (value <=0):
        return "red"
    elif (value <= 3):
        return "orange"
    else:
        return "green"

def getWealth6(df):
    df['wealth'] = df['livestock_number_gosh'].map(score_gs) + df['livestock_number_cattle'].map(score_cattle)
    return df['wealth']
    
prueba6 = pd.crosstab(livestock_set.livestock_number_gosh, livestock_set.livestock_number_cattle).stack().reset_index(name='number')
#prueba.plot.scatter('livestock_number_goats', 'livestock_number_sheep', s=prueba.number * 10)
prueba6 = prueba6[prueba6.number != 0]
prueba6= prueba6.drop(0)
#prueba5 = prueba5[prueba5.livestock_number_gosh <= 100]
#prueba5 = prueba5[prueba5.livestock_number_camel <= 100]
#display(prueba)
gosh_vs_cattle = {
    "layout": {
        "title": "Relation between the number of camel and sheeps",
        "xaxis": {
            "title": "Number of goat"
        },
        "yaxis": {
            "title": "Number of camel"
        }
    },
    "data": [
        {

            "x": prueba6.livestock_number_gosh,
            "y": prueba6.livestock_number_cattle,
            "mode": "markers",
            "marker": {
                "size": prueba6.number * 5,
                "color": getWealth6(prueba6).map(getColor6)
            },
            "type": "scatter"
        }
    ]
}
iplot(gosh_vs_cattle)

With a few exceptions, the number of goats per household is proportional to the number of sheeps. So to define wealth groups for livestock tenants the number of goats will be the main variable.

In [43]:
goats = goats.value_counts().sort_index()
goats_femHoHH = goats_femHoHH.value_counts().sort_index()
sheeps = sheeps.value_counts().sort_index()
sheeps_femHoHH = sheeps_femHoHH.value_counts().sort_index()
cattle = cattle.value_counts().sort_index()
cattle_femHoHH = cattle_femHoHH.value_counts().sort_index()

In [44]:
values = [4,15, 80]
print values
values_femHoHH = [5,10, 40]
print values_femHoHH
per2 = get_percentiles(goats, values)
percentiles = pd.Series(values, index = per2)

print percentiles

print pd.Series(values_femHoHH, index = get_percentiles(goats_femHoHH, values_femHoHH))

plot_wealth_criteria(goats, values, "Number of goats per wealth group")
#plot_wealth_criteria(land_quantity_female, values_female, "Number of feddans per wealth group in female respondents")
plot_wealth_criteria(goats_femHoHH, values_femHoHH, "Number of goats per wealth group in female HoHH")

[4, 15, 80]
[5, 10, 40]
0.18     4
0.58    15
0.96    80
dtype: int64
0.238095     5
0.523810    10
0.857143    40
dtype: int64
5.0      3.0
6.0      3.0
7.0      2.0
8.0      4.0
10.0    15.0
12.0     2.0
15.0    11.0
Name: livestock_number_goat, dtype: float64


6.0      4.761905
8.0      4.761905
10.0    19.047619
Name: livestock_number_goat, dtype: float64


In [45]:
values = [4,15, 80]
print values
values_femHoHH = [0,5, 20]
print values_femHoHH
per2 = get_percentiles(sheeps, values)
percentiles = pd.Series(values, index = per2)

print percentiles

print pd.Series(values_femHoHH, index = get_percentiles(sheeps_femHoHH, values_femHoHH))

plot_wealth_criteria(sheeps, values, "Number of sheeps per wealth group")
#plot_wealth_criteria(land_quantity_female, values_female, "Number of feddans per wealth group in female respondents")
plot_wealth_criteria(sheeps_femHoHH, values_femHoHH, "Number of sheeps per wealth group in female HoHH")

[4, 15, 80]
[0, 5, 20]
0.36     4
0.57    15
0.89    80
dtype: int64
0.238095     0
0.333333     5
0.714286    20
dtype: int64
5.0     3.0
6.0     3.0
7.0     1.0
8.0     1.0
10.0    6.0
12.0    1.0
15.0    6.0
Name: livestock_number_sheep, dtype: float64


3.0    4.761905
5.0    4.761905
Name: livestock_number_sheep, dtype: float64


#### Quantiles for goats
- Poors: 35% of households have less than 10 goats and less than 10 sheeps
- Medium: 62% of households have between 10 and 35 goats and between 20 and 30 sheeps
- Richs: 3% of households have more than 35 goats and/or more than 30 sheeps

In [46]:
livestock_uses = ["selling", "social", "milk", "meat", "savings"]
goats_uses = ["selling", "social", "milk", "meat", "savings"]
sheeps_uses = ["selling", "social", "milk", "meat", "savings"]
livestock_uses_df = livestock_set[map(lambda x: "livestock_" + x, livestock_uses)]
livestock_uses_df.fillna('no')
livestock_uses = livestock_uses_df.apply(pd.Series.value_counts).fillna('0').loc['yes']
num_lvstck = livestock_set['deviceid'].count()
livestock_uses_perc = livestock_uses * 100 / num_lvstck

fig_uses_lvstck = {
    "layout": {
        "title": "Livestock uses",
        "xaxis": {
            "tickangle": -45
        },
        "yaxis": {
            "ticksuffix": "%"
        },
        "width": 700,
        "annotations": []
    },
    "data": [
        {
            "x": map(lambda x: x.capitalize(), livestock_uses_perc.index.values),
            "y": livestock_uses_perc.values,
            "type": "bar",
            "name": "Live uses",
            "marker": {
                "color": 'rgb(258,225,225)',
                "line": {
                    "color": 'rgb(192,0,0)',
                    "width": 1.5
                }
            }
        }
    ]
}


iplot(fig_uses_lvstck)

In [47]:
print "USES TOTAL LIVESTOCK"
print livestock_uses
print livestock_uses_perc
goats_uses = livestock_set.loc[livestock_set['livestock_type'].str.contains("goat")][map(lambda x: "livestock_" + x, goats_uses)]
print ""
print "USES TOTAL GOATS"
print goats_uses.apply(pd.Series.value_counts)
sheeps_uses = livestock_set.loc[livestock_set['livestock_type'].str.contains("sheep")][map(lambda x: "livestock_" + x, sheeps_uses)]
print ""
print "USES TOTAL SHEEPS"
print sheeps_uses.apply(pd.Series.value_counts)

USES TOTAL LIVESTOCK
livestock_selling    96
livestock_social     33
livestock_milk       97
livestock_meat       70
livestock_savings    97
Name: yes, dtype: object
livestock_selling    96
livestock_social     33
livestock_milk       97
livestock_meat       70
livestock_savings    97
Name: yes, dtype: object

USES TOTAL GOATS
           livestock_selling  livestock_social  livestock_milk  \
no                         3              60.0             1.0   
no_answer                  1               NaN             NaN   
yes                       85              29.0            88.0   

           livestock_meat  livestock_savings  
no                   28.0                2.0  
no_answer             NaN                NaN  
yes                  61.0               87.0  

USES TOTAL SHEEPS
           livestock_selling  livestock_social  livestock_milk  \
no                         2              43.0             2.0   
no_answer                  1               NaN             NaN   
y

In [48]:
uses_lvstck = get_ocurrences(livestock_set, 'livestock_type')

perc_lvstck = 100*types_lvstck/livestock_set['deviceid'].count()
perc_lvstck = perc_lvstck.round(2)
fig_types_lvstck = {
    "layout": {
        "title": "Livestock types",
        "xaxis": {
            "tickangle": -45
        },
        "width": 700,
        "annotations": []
    },
    "data": [
        {
            "x": map(lambda x: x.capitalize(), perc_lvstck.index.values),
            "y": perc_lvstck.values,
            "type": "bar",
            "name": "Livelihoods Strategies",
            "marker": {
                "color": 'rgb(258,225,225)',
                "line": {
                    "color": 'rgb(192,0,0)',
                    "width": 1.5
                }
            }
        }
    ]
}


for i in range(0, len(types_lvstck)):
    fig_types_lvstck["layout"]["annotations"].append(
        {
            "x": map(lambda x: x.capitalize(), perc_lvstck.index.values)[i],
            "y": perc_lvstck.values[i] + 1.5,
            "text": str(perc_lvstck.values[i]) + "%",
            "font": dict(
                family='Arial',
                size=14,
                color='rgba(100, 0, 0, 1)'
            ),
            "showarrow": False,

        }
    )

iplot(fig_types_lvstck)

### Laborers

In [49]:
labour_set = reg_survey.loc[reg_survey['src_incomes'].str.contains("labour")]
labour_set_femHoHH = reg_survey_femHoHH.loc[reg_survey_femHoHH['src_incomes'].str.contains("labour")]
labor_sector = get_ocurrences(labour_set, 'labour_sector')
labor_sector_femHoHH = get_ocurrences(labour_set_femHoHH, 'labour_sector')
#create_pie_graph(labor_sector, "Labor sector")
create_graphs_HoHH(lambda x: get_ocurrences(x.loc[x['src_incomes'].str.contains("labour")], 'labour_sector'),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Labor sector")
labour_set.loc[labour_set['labour_participants'].str.contains('women')][['labour_participants', 'respondant_gender', 'respondant_age', 'respondant_head_hh', 'male_teenager', 'male_adult', 'male_elderly', 'labour_sector']]
labour_incomes = labour_set.groupby('labour_daily_salary').size()
mean_salary = labour_set['labour_daily_salary'].mean()
sectors_100 = labour_set.loc[labour_set['labour_daily_salary'] == 100].groupby('labour_sector').size()
print str(labour_set['deviceid'].count()) + " laborers"
create_graphs_HoHH(lambda x: x.loc[x['src_incomes'].str.contains("labour")].groupby('labour_work_term').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Temporality of laborers")
create_graphs_HoHH(lambda x: x.loc[x['src_incomes'].str.contains("labour")].groupby('labour_employment_type').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Employment Type")
create_graphs_HoHH(lambda x: x.loc[x['src_incomes'].str.contains("labour")].groupby('labour_participants').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Gender of laborers")

104 laborers


In [50]:
gov_salaries = labour_set.loc[labour_set['labour_sector'] == 'government']
agri_salaries = labour_set.loc[labour_set['labour_sector'] == 'agriculture']
trans_salaries = labour_set.loc[labour_set['labour_sector'] == 'transportation']
other_salaries = labour_set.loc[labour_set['labour_sector'] == 'other']
print mean_salary
gov_salaries_mean = gov_salaries['labour_daily_salary'].mean()
gov_salaries_num = gov_salaries.groupby('labour_daily_salary').size()
agri_salaries_mean = agri_salaries['labour_daily_salary'].mean()
agri_salaries_num = agri_salaries.groupby('labour_daily_salary').size()
trans_salaries_mean = trans_salaries['labour_daily_salary'].mean()
trans_salaries_num = trans_salaries.groupby('labour_daily_salary').size()
other_salaries_mean = other_salaries['labour_daily_salary'].mean()
other_salaries_num = other_salaries.groupby('labour_daily_salary').size()

salaries = {
    "layout": {
        "title": "Relation between the number of cattle and sheeps",
        "xaxis": {
            "title": "Number of goat"
        },
        "yaxis": {
            "title": "Number of cattle"
        }
    },
    "data": [
        {

            "x": prueba3.livestock_number_goat,
            "y": prueba3.livestock_number_cattle,
            "mode": "markers",
            "marker": {
                "size": prueba.number * 5               
            },
            "type": "scatter"
        }
    ]
}
iplot(goat_vs_cattle)

54.6634615385


### Agriculture

In [51]:
agri_set = reg_survey.loc[reg_survey['src_incomes'].str.contains("agriculture")].copy()
agri_set_female = reg_survey_female.loc[reg_survey_female['src_incomes'].str.contains("agriculture")]#.copy()
agri_set_femHoHH = reg_survey_femHoHH.loc[reg_survey_femHoHH['src_incomes'].str.contains("agriculture")].copy()

land_quantity = agri_set.groupby('agri_gard_land_quantity').size()
land_quantity_female = agri_set_female.groupby('agri_gard_land_quantity').size()
land_quantity_femHoHH = agri_set_femHoHH.groupby('agri_gard_land_quantity').size()
use_quantity = agri_set.groupby('agri_gard_use_quantity').size()

d = {'land': land_quantity, 'use': use_quantity}
use_land = pd.DataFrame(d)
hey = use_quantity.index.map(lambda x: str(x))
use_quantity.index = hey
print use_quantity.index
display(use_quantity)
display(use_land)
display(land_quantity)

Index([u'0.0', u'1.0', u'2.0', u'3.0', u'4.0', u'5.0', u'6.0', u'8.0', u'10.0',
       u'15.0', u'20.0', u'24.0', u'30.0', u'34.0', u'40.0', u'50.0', u'60.0',
       u'100.0', u'150.0', u'200.0', u'360.0'],
      dtype='object', name=u'agri_gard_use_quantity')


agri_gard_use_quantity
0.0       2
1.0       3
2.0      10
3.0       2
4.0       3
5.0       5
6.0       2
8.0       1
10.0      7
15.0      1
20.0      3
24.0      1
30.0      2
34.0      1
40.0      1
50.0      2
60.0      3
100.0     4
150.0     1
200.0     1
360.0     1
dtype: int64

,land,use
0.0,7.0,2.0
1.0,2.0,3.0
2.0,9.0,10.0
3.0,3.0,2.0
4.0,3.0,3.0
5.0,6.0,5.0
6.0,2.0,2.0
7.0,1.0,NaN
8.0,NaN,1.0
10.0,5.0,7.0


agri_gard_land_quantity
0.0      7
1.0      2
2.0      9
3.0      3
4.0      3
5.0      6
6.0      2
7.0      1
10.0     5
15.0     2
20.0     3
30.0     2
34.0     1
40.0     1
50.0     1
60.0     2
100.0    2
150.0    1
200.0    1
250.0    1
300.0    1
360.0    1
dtype: int64

In [52]:
values = [0,5, land_quantity.index[-4]]
values_female = [0,5, land_quantity_female.index[-3]]
values_femHoHH = [0,5, land_quantity_femHoHH.index[-2]]
per2 = get_percentiles(land_quantity, values)
percentiles = pd.Series(values, index = per2)

print percentiles

print pd.Series(values_female, index = get_percentiles(land_quantity_female, values_female))
print pd.Series(values_femHoHH, index = get_percentiles(land_quantity_femHoHH, values_femHoHH))

plot_wealth_criteria(land_quantity, values, "Number of feddans per wealth group")
#plot_wealth_criteria(land_quantity_female, values_female, "Number of feddans per wealth group in female respondents")
plot_wealth_criteria(land_quantity_femHoHH, values_femHoHH, "Number of feddans per wealth group in female HoHH")

0.122807      0.0
0.526316      5.0
0.947368    200.0
dtype: float64
0.2     0.0
0.6     5.0
0.8    10.0
dtype: float64
0.000000     0.0
0.500000     5.0
0.833333    40.0
dtype: float64
agri_gard_land_quantity
1.0     3.508772
2.0    15.789474
3.0     5.263158
4.0     5.263158
5.0    10.526316
dtype: float64


agri_gard_land_quantity
2.0    16.666667
5.0    33.333333
dtype: float64


In [53]:
quantiles = [.335, .915, 1]
q = goats.quantile(quantiles)
print "Quantiles:"
print q
goats1 = goats[goats < q[quantiles[0]]].dropna()
goats2 = goats[goats.between(q[quantiles[0]], q[quantiles[1]])].dropna()
goats3 = goats[goats.between(q[quantiles[1]], q[quantiles[2]])].dropna()
#fig = ff.create_distplot([goats1.values, goats2.values, goats3.values], ['poor', 'medium', 'rich'])
#iplot (fig)

Quantiles:
0.335     1.0
0.915    11.0
1.000    21.0
Name: livestock_number_goat, dtype: float64


In [54]:
use_vs_have = pd.crosstab(agri_set.agri_gard_land_quantity, agri_set.agri_gard_use_quantity).stack().reset_index(name='feddans')
#prueba.plot.scatter('livestock_number_goats', 'livestock_number_sheep', s=prueba.number * 10)
use_vs_have = use_vs_have[use_vs_have.feddans != 0]
use_vs_have = use_vs_have[use_vs_have.agri_gard_use_quantity <= 10]
use_vs_have = use_vs_have[use_vs_have.agri_gard_land_quantity <= 10]
#prueba = prueba[prueba.livestock_number_goat <= 100]
#prueba = prueba[prueba.livestock_number_sheep <= 100]
#display(prueba)
use_vs_have = {
    "layout": {
        "title": "Relation between the quantity of land owned and used"
    },
    "data": [
        {

            "x": use_vs_have.agri_gard_land_quantity,
            "y": use_vs_have.agri_gard_use_quantity,
            "mode": "markers",
            "marker": {
                "size": use_vs_have.feddans
            },
            "type": "scatter"
        },
        {

            "x": [0,10],
            "y": [0,10],
            "mode": "lines"
        }
    ]
}
iplot(use_vs_have)

In [55]:
# Inconsistencies
# print agri_set.loc[agri_set.agri_gard_water_harv_farmlands > agri_set.agri_gard_land_quantity]['agri_gard_water_harv_farmlands']
# print agri_set.loc[agri_set.agri_gard_water_harv_farmlands > agri_set.agri_gard_land_quantity]['agri_gard_land_quantity']

agri_set['perc_rainfed_farmlands'] = agri_set['agri_gard_water_harv_farmlands'] / agri_set['agri_gard_land_quantity']

# Estimation to correct the inconsistencies: All feddan have water harvesting access
feddans_med_poor = agri_set.loc[agri_set.agri_gard_use_quantity <= 5]['agri_gard_use_quantity'].sum()

print feddans_med_poor / agri_set.loc[agri_set.agri_gard_use_quantity <= 5, 'deviceid'].count()
print feddans_med_poor / total_surveys

#print agri_set.groupby('perc_rainfed_farmlands').size()
#print agri_set.groupby('agri_gard_farm_tech').size()
#print agri_set.groupby('agri_gard_water_harv_farmlands').size()
all_farm_rainfed = agri_set.loc[agri_set['perc_rainfed_farmlands'] == 1].groupby('agri_gard_use_quantity').size()
agri_set['agri_gard_water_harv_farmlands'].sum()
print agri_set.loc[agri_set.agri_gard_use_quantity > agri_set.agri_gard_land_quantity]['deviceid'].count()
print agri_set.loc[agri_set.agri_gard_land_property == 'own'].groupby('agri_gard_use_quantity').size()
print agri_set.loc[agri_set.agri_gard_land_property == 'rent'].groupby('agri_gard_use_quantity').size()
print agri_set.groupby('agri_gard_land_quantity').size()
print agri_set.groupby('cc_effects_sand_affected_farmlands').size()

agri_set.groupby('agri_gard_training').size()

2.64
0.304147465438
6
agri_gard_use_quantity
2.0      7
3.0      1
4.0      2
6.0      2
8.0      1
10.0     1
15.0     1
30.0     1
34.0     1
40.0     1
50.0     1
100.0    3
150.0    1
200.0    1
360.0    1
dtype: int64
agri_gard_use_quantity
0.0     2
1.0     1
2.0     1
10.0    5
24.0    1
60.0    3
dtype: int64
agri_gard_land_quantity
0.0      7
1.0      2
2.0      9
3.0      3
4.0      3
5.0      6
6.0      2
7.0      1
10.0     5
15.0     2
20.0     3
30.0     2
34.0     1
40.0     1
50.0     1
60.0     2
100.0    2
150.0    1
200.0    1
250.0    1
300.0    1
360.0    1
dtype: int64
cc_effects_sand_affected_farmlands
0.0      24
1.0      18
2.0       2
3.0       1
5.0       2
7.0       1
10.0      3
30.0      2
50.0      2
70.0      1
160.0     1
dtype: int64


agri_gard_training
no    57
dtype: int64

In [56]:
agri_set_med_poor = agri_set.loc[agri_set.agri_gard_use_quantity <= 5].copy()
agri_set_med_poor.loc[agri_set_med_poor['agri_gard_protected_farmlands'] > agri_set_med_poor['agri_gard_use_quantity'], 'agri_gard_protected_farmlands'] = agri_set_med_poor['agri_gard_use_quantity']
agri_set_med_poor['perc_land_protected'] = agri_set_med_poor.agri_gard_protected_farmlands / agri_set_med_poor.agri_gard_land_quantity
print agri_set_med_poor.groupby('agri_gard_use_quantity').size()
print agri_set_med_poor['agri_gard_protected_farmlands'].sum() / agri_set_med_poor['agri_gard_use_quantity'].sum()
print agri_set_med_poor.groupby('perc_land_protected').size()
print agri_set_med_poor.groupby('perc_land_protected').size().sum()

agri_gard_use_quantity
0.0     2
1.0     3
2.0    10
3.0     2
4.0     3
5.0     5
dtype: int64
0.121212121212
perc_land_protected
0.0    13
0.2     3
1.0     2
dtype: int64
18


In [57]:
#crop_types = ['sorghum', 'wheat', 'bean', 'palm', 'fodder', 'watermelon', 'potato', 'tomato', 'onion', 'cucumber', 'okra', 'other']

types_agri = get_ocurrences(agri_set, 'agri_gard_agri_crops')
types_agri = types_agri / agri_set['deviceid'].count()
print types_agri

crops = {
    "layout": {
        "title": "Main crops",
        "xaxis": {
            "tickangle": -45
        },
        "width": 700,
        "annotations": []
    },
    "data": [
        {
            "x": map(lambda x: x.capitalize(), types_agri.index.values),
            "y": types_agri.values,
            "type": "bar",
            "name": "Livelihoods Strategies",
            "marker": {
                "color": 'rgb(205,258,205)',
                "line": {
                    "color": 'rgb(0,192,0)',
                    "width": 1.5
                }
            },
        }
    ]
}

iplot(crops)

sorghum    1.000000
fodder     0.017544
tomato     0.192982
okra       0.245614
millet     0.017544
other      0.105263
dtype: float64


In [58]:
farm_tools = reg_survey.groupby('agri_gard_farm_tools').size()
farm_tech = reg_survey.groupby('agri_gard_farm_tech').size()
seeds = reg_survey.groupby('agri_gard_seeds').size()

create_graphs_HoHH(lambda x: x.groupby('agri_gard_farm_tools').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Farming tools by Head of Household")

create_graphs_respondent(lambda x: x.groupby('agri_gard_farm_tools').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Farming tools by respondent")

In [59]:
create_graphs_HoHH(lambda x: x.groupby('agri_gard_farm_tech').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Farming tools by Head of Household")

create_graphs_respondent(lambda x: x.groupby('agri_gard_farm_tech').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Farming tools by respondent")

In [60]:
create_graphs_HoHH(lambda x: x.groupby('agri_gard_seeds').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Farming tools by Head of Household")

create_graphs_respondent(lambda x: x.groupby('agri_gard_seeds').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Farming tools by respondent")

In [61]:
crops = get_ocurrences(agri_set, 'agri_gard_agri_crops')
print crops

sorghum    57
fodder      1
tomato     11
okra       14
millet      1
other       6
dtype: int64


### Forestry (charcoal production)

In [62]:
forestry_set = reg_survey.loc[reg_survey['src_incomes'].str.contains("forestry")]
print forestry_set.groupby('forestry_forestry_type').size()
print get_ocurrences(forestry_set, 'forestry_forestry_type')
forestry_participants =  forestry_set.groupby('forestry_participants').size()
#forestry_set.loc[forestry_set['forestry_participants'].str.contains('women')][['forestry_participants', 'respondant_gender', 'respondant_age', 'respondant_head_hh', 'male_teenager', 'male_adult', 'male_elderly', 'labour_sector']]
forestry_land_property = forestry_set.groupby('forestry_land_property').size()

create_pie_graph(forestry_set.groupby('forestry_forestry_type').size(), "Forestry Type")
create_pie_graph(forestry_set.groupby('forestry_participants').size(), "Forestry participants")
create_pie_graph(forestry_set.groupby('forestry_land_property').size(), "Forests property")

print forestry_participants
print forestry_land_property

forestry_forestry_type
charcoal fuelwood    1
dtype: int64
charcoal    1
fuelwood    1
dtype: int64


forestry_participants
men    1
dtype: int64
forestry_land_property
none    1
dtype: int64


In [63]:
def create_2pie_graph(serie1, serie2, title, title1, title2):
    fig = {
        "layout": {
            "title": title,
            "annotations": [
                {
                    "font": {
                        "size": 14
                    },
                    "showarrow": False,
                    "text": title1,
                    "x": .20,
                    "y": 1.1
                },
                {
                    "font": {
                        "size": 14
                    },
                    "showarrow": False,
                    #"text": title2 + " (" + str(sum_func(data_global)) + ")",
                    "text": title2,
                    "x": .8,
                    "y": 1.1 
                }
            ]
        },
        "data": [
            {
                "values": serie1.values,
                "labels": serie1.index.values,
                "type": "pie",
                "sort": False,
                "domain": {
                    "x": [.07, .43]
                },
            },
            {
                "values": serie2.values,
                "labels": serie2.index.values,
                "type": "pie",
                "sort": False,
                "domain": {
                    "x": [.57, .93]
                },
            }
        ]
    }

    iplot(fig)
    
create_2pie_graph(labour_set.groupby('labour_training').size(), livestock_set.groupby('livestock_training').size(), "Population who has received training in their livelihoods strategies", "Labour", "Livestock")

### Other

In [64]:
other_set = reg_survey.loc[reg_survey['src_incomes'].str.contains("other")]
other_set.groupby('other_training').size()


other_training
no     37
yes     2
dtype: int64

### Coping strategies

In [65]:
cop_strats = get_ocurrences(reg_survey, 'coping_strat')
cop_strats = cop_strats * 100 / total_surveys

fig_cop_strats = {
    "layout": {
        "title": "Coping Strategies",
        "xaxis": {
            "tickangle": -45
        },
        "yaxis": {
            "ticksuffix": "%"
        },
        "width": 700,
        "annotations": []
    },
    "data": [
        {
            "x": map(lambda x: x.capitalize(), cop_strats.index.values),
            "y": cop_strats.values,
            "type": "bar",
            "name": "Livelihoods Strategies",
            "marker": {
                "color": 'rgb(205,205,258)',
                "line": {
                    "color": 'rgb(0,0,192)',
                    "width": 1.5
                }
            },
        }
    ]
}

iplot(fig_cop_strats)

## Climate change effects

In [66]:
cc_effects_on_rain = get_ocurrences(reg_survey, 'changed_rainy_season')

print cc_effects_on_rain
create_pie_graph(cc_effects_on_rain, "Respondents asked about rain changes in last ten years")

decreased        178
increased         35
no_changes         2
unpredictable      3
unsteady           1
dtype: int64


In [67]:
sand_mov_sums = reg_survey.groupby('sand_movements').size()
wind_changes = get_ocurrences(reg_survey, 'sand_movements')

print wind_changes
print sand_mov_sums

create_graphs_respondent(lambda x: x.groupby('sand_movements').size(),
                         lambda x: x.values,
                         lambda x: x.index.values,
                         lambda x: x.sum(),
                         "Respondents asked about wind changes in las 5 years")

create_pie_graph(wind_changes, "Respondents asked about wind changes in last 5 years")

less_windy        53
more_windy       108
no_changes         7
unpredictable     51
dtype: int64
sand_movements
less_windy                   52
more_windy                  106
more_windy less_windy         1
no_changes                    7
unpredictable                50
unpredictable more_windy      1
dtype: int64


In [68]:
agri_set_med_poor.loc[agri_set_med_poor['cc_effects_sand_affected_farmlands'] > agri_set_med_poor['agri_gard_use_quantity'], 'cc_effects_sand_affected_farmlands'] = agri_set_med_poor['agri_gard_use_quantity']
agri_set_med_poor['perc_land_affected'] = agri_set_med_poor.cc_effects_sand_affected_farmlands / agri_set_med_poor.agri_gard_land_quantity
print agri_set_med_poor['cc_effects_sand_affected_farmlands'].sum() / agri_set_med_poor['agri_gard_use_quantity'].sum()
print agri_set_med_poor.groupby('perc_land_affected').size()
print agri_set_med_poor.groupby('perc_land_affected').size().sum()

0.363636363636
perc_land_affected
0.000000    13
0.200000     2
0.333333     1
0.500000     2
1.000000     6
dtype: int64
24


In [69]:
temps_changes = get_ocurrences(reg_survey, 'temps_change')

print temps_changes

decreased         85
increased        117
unpredictable     10
no_changes         4
unsteady           2
dtype: int64


In [70]:
effects_on_lvstck = get_ocurrences(reg_survey, 'cc_effects_grassland_availability').sort_values(ascending = False)
lvstck_perc = 100*effects_on_lvstck/livestock_set['deviceid'].size
lvstck_perc = lvstck_perc.round(2)
fig_effects_on_lvstck = {
    "layout": {
        "title": "Trend of the availability of grassland in the last 5 years",
        "xaxis": {
            "tickangle": -45
        },
        "width": 700,
        "annotations": []
    },
    "data": [
        {
            "x": map(lambda x: x.capitalize(), lvstck_perc.index.values),
            "y": lvstck_perc.values,
            "type": "bar",
            "marker": {
                "color": 'rgb(25,158,25,0.6)',
                "line": {
                    "color": 'rgb(0,70,0)',
                    "width": 1.5
                }
            },
        }
    ]
}


for i in range(0, len(effects_on_lvstck)):
    fig_effects_on_lvstck["layout"]["annotations"].append(
        {
            "x": map(lambda x: x.capitalize(), lvstck_perc.index.values)[i],
            "y": lvstck_perc.values[i] + 1.7,
            "text": str(lvstck_perc.values[i]) + "%",
            "font": dict(
                family='Arial',
                size=14,
                color='rgba(0, 100, 0, 1)'
            ),
            "showarrow": False,

        }
    )

iplot(fig_effects_on_lvstck)
